# Example

In [1]:
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [3]:
y_test

array([1, 1, 1, 1, 1, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 1, 1, 0, 0, 2,
       0, 0, 0, 0, 0, 2, 2, 0])

In [2]:
preds

array([1, 1, 1, 1, 1, 0, 0, 2, 0, 2, 0, 0, 1, 2, 2, 0, 2, 1, 1, 0, 0, 2,
       0, 0, 0, 0, 0, 2, 2, 0], dtype=int64)

# Setup

# Data Reading